In [4]:
import pandas as pd
import gezi
from gezi import tqdm

In [5]:
dcs = []
for k in tqdm(range(30)):
  d = gezi.read_parquet(f'../input/train/part_{k + 1}/context.parquet')
  d['day'] = k + 1
  dcs += [d]
d = gezi.read_parquet('../input/eval/context.parquet')
d['day'] = 31
dcs += [d]
dc = pd.concat(dcs)

100%|██████████| 30/30 [00:51<00:00,  1.48s/it]


In [65]:
for i in tqdm(range(30)):
#   d = dcs[i].groupby('did').agg({'label':lambda x: list(x), 'vid': lambda x: list(x)}).reset_index()
#   d['num_shows'] = d.vid.apply(lambda x: len(json.loads(x)))
  d = dcs[i].groupby('did').agg(labels=('label', lambda x: list(x)), vids=('vid', lambda x: list(x))).reset_index()
  d['num_shows'] = d.vids.apply(len)
  d.to_csv(f'../input/train/part_{i + 1}/user_shows.csv', index=False)

100%|██████████| 30/30 [17:06<00:00, 38.82s/it]


In [91]:
d = dcs[30].groupby('did').agg(vids=('vid', lambda x: list(x))).reset_index()
# d['num_shows'] = d.vid.apply(lambda x: len(json.loads(x)))
d['num_shows'] = d.vids.apply(len)
d.to_csv(f'../input/eval/user_shows.csv', index=False)

In [ ]:
!ls ../in

In [92]:
d = pd.read_csv('../input/eval/user_shows.csv')

In [93]:
d

,did,vids,num_shows
0,000076ac36f445652aa5e4c852b0fc60,"[928950721, 3943801933, 3463293293, 2748791550...",19
1,0000e0766fb3a0fbf315141a4cd75ba9,"[3783061137, 1754700222, 960875967, 1589744395...",21
2,000112736d8c493b432aa96d91eb7a05,"[869074554, 137583840, 288470582, 3039103562, ...",30
3,00017aebaec28ab239fef9b9c203b201,"[1117575107, 3592796172, 2445992384, 3295574875]",4
4,0002141519dfa883d0aef37ecbfbb826,"[2822509363, 262027222, 203928561, 951842254, ...",15
...,...,...,...
156268,fffece9cbfa6e09bc57fade767cd9af4,"[1255606946, 3299736599, 218566039, 1653383727...",14
156269,fffede667adcee2bd3349d5b0d93cb72,"[1589744395, 3247800388, 726123253, 3968659030...",8
156270,ffff6e42c30054bbb80f12487ae2f1c4,"[4119730190, 3551221584, 4153053381, 351923321...",43
156271,ffffdf17558c411767561b3d0160e5c5,"[1295103041, 1316999893, 1087487841, 190642780...",72


In [83]:
d['x'] = d.vids.apply(json.loads)

In [86]:
d.x.isnull().mean()

0.0

In [94]:
udf = gezi.read_parquet('../input/eval/user.parquet')

In [95]:
x = udf.merge(d, how='left', on='did')

In [96]:
x

,did,watch,vids,num_shows
0,000c3ba80c2409c25397c3b7a96bf7b7,"[[1426584338, 1689874036], [1426582429, 396537...","[4153053381, 1803697316, 392040838, 2066299152...",37
1,002935918fb49444e22021a444bed3ef,"[[1427039460, 2168299795], [1427022514, 205222...","[3696459075, 928950721, 4117691755, 2259761821...",86
2,005520bbde1b22072e0f5a2a0de90307,"[[1426920166, 2338506193], [1426918447, 228034...","[1845994138, 3974683372, 877648723, 353146523,...",9
3,0128d5011e88e088c23a6ac00aa74250,"[[1426942728, 3650366537], [1426942632, 108342...","[2525377531, 3465302719, 2788324596, 129021576...",76
4,0208e13c29cb5cf22ce4c5dab56b8a5a,"[[1426955612, 1083423540], [1426955512, 119057...","[1139644873, 1116119361, 1814801369, 357102979...",16
...,...,...,...,...
156268,feeb8c669a7cd6f5810fecbe0c9815f1,"[[1427036845, 78776638], [1427036039, 16533837...","[3000443306, 4119730190, 2045976749, 4265770661]",4
156269,feef89c4d16cf2fdfbd42f1b82ece2c7,"[[1426566573, 3853814243], [1426566205, 164466...","[3702731792, 1007156689, 2663868212, 222704907...",54
156270,ff13fa73325f60e61a2a353f9ca7018b,"[[1426860075, 247533023], [1426860038, 1817923...","[2713919673, 2036339327, 2748791550, 177231369...",5
156271,ff82b8247eaf6a9fe24235985c3215cb,"[[1426680642, 4160178488], [1426677797, 123791...","[2680807466, 2600015807, 1648652799, 394380193...",30


In [78]:
type(d.vids.apply(json.loads)[0][0])

int

In [42]:
import json
d['num_shows'] = d.vid.apply(lambda x: len(json.loads(x)))

In [43]:
d

,did,vid,num_shows
0,000076ac36f445652aa5e4c852b0fc60,"[928950721, 3943801933, 3463293293, 2748791550...",19
1,0000e0766fb3a0fbf315141a4cd75ba9,"[3783061137, 1754700222, 960875967, 1589744395...",21
2,000112736d8c493b432aa96d91eb7a05,"[869074554, 137583840, 288470582, 3039103562, ...",30
3,00017aebaec28ab239fef9b9c203b201,"[1117575107, 3592796172, 2445992384, 3295574875]",4
4,0002141519dfa883d0aef37ecbfbb826,"[2822509363, 262027222, 203928561, 951842254, ...",15
...,...,...,...
156268,fffece9cbfa6e09bc57fade767cd9af4,"[1255606946, 3299736599, 218566039, 1653383727...",14
156269,fffede667adcee2bd3349d5b0d93cb72,"[1589744395, 3247800388, 726123253, 3968659030...",8
156270,ffff6e42c30054bbb80f12487ae2f1c4,"[4119730190, 3551221584, 4153053381, 351923321...",43
156271,ffffdf17558c411767561b3d0160e5c5,"[1295103041, 1316999893, 1087487841, 190642780...",72


In [44]:
d.num_shows.max()

1510

In [51]:
d[d.num_shows > 500]

,did,vid,num_shows
415,00c327f379e2ed1ba73f108d3b5c78ed,"[3163421291, 2217700563, 687921821, 1204634681...",527
1690,02db2ac2f038ce36c7ba9bb95b72a21f,"[1917445238, 3324599518, 3601195672, 413318357...",1044
4219,071175d8fa53b9999eb358e8275180c3,"[600057505, 515725709, 1529173679, 1646633441,...",536
5787,09b2b3117c8ba8cc20c71a3e2b354d2c,"[1590371954, 1795025585, 1189434277, 403994937...",923
7466,0c732ae9d97f09f4b0a35e33734d9294,"[2923347975, 2060480076, 305323132, 61393195, ...",562
...,...,...,...
152106,f9453519b2232b77c81b46f4c9661929,"[1148279038, 3999366731, 3160633545, 235640982...",571
152716,fa3acfb17da02f61418cd743281189f1,"[810899268, 354718091, 2626837309, 3861880248,...",506
154511,fd2f028bad025d375ca779c8f96c0fde,"[1012031629, 988875113, 2695107668, 1415018885...",960
155268,fe6862eeb15cc46900caeaf60eab65c6,"[1383998271, 2828410770, 4239008993, 356605126...",506


In [49]:
d.num_shows.describe()

count    156273.000000
mean         35.526495
std          51.077872
min           2.000000
25%          10.000000
50%          20.000000
75%          41.000000
max        1510.000000
Name: num_shows, dtype: float64

In [97]:
d = pd.read_csv('../input/train/part_1/user_shows.csv')

In [98]:
d.num_shows.describe()

count    75017.000000
mean        42.766919
std         62.568084
min          2.000000
25%         11.000000
50%         24.000000
75%         49.000000
max       1978.000000
Name: num_shows, dtype: float64

In [25]:
d[d.num_shows > 100]

,label,vid,num_shows
22,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[3837399416, 1518464850, 3834038755, 231452705...",213
35,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[2550491746, 663772741, 3924895537, 3052462695...",119
44,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[2839104658, 1911351853, 1684974379, 325186854...",363
51,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1241251750, 598970505, 1685051940, 3900506745...",212
71,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[2839104658, 460107879, 1024514146, 454895160,...",164
...,...,...,...
74873,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, ...","[3966837672, 1684974379, 4109864961, 129105148...",194
74944,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[3966837672, 3462130625, 1249671629, 198765473...",111
74984,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[2839104658, 2137624582, 3554499313, 232449567...",106
74987,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[3966837672, 3241416253, 2847552933, 341235370...",172


In [33]:
d = dcs[29].sample(frac=0.2)

In [30]:
len(dcs[29])

3324513

In [31]:
d[['vids', '']] = d.groupby('did').agg({'label':lambda x: list(x), 'vid': lambda x: list(x)}

In [34]:
d

,label,mod,mf,aver,sver,did,vid,prev,region,timestamp,index,day
163083,0,bdc2f68f307193371423123a1f8b9d38,fce8cc8ce0587de13c3fecbc13537746,58655385700ba140fd860b23e2e1e9f6,e49dc1e8e576ff35a07e1182f4e98c91,33c543ae7f81b60b299ffae5981f6798,4270158929,0,1758996089,1426921151,163083,30
680054,0,07c54d4eecfcccbb0503f4cc6b672741,fce8cc8ce0587de13c3fecbc13537746,58655385700ba140fd860b23e2e1e9f6,21032c3a66d86aa1d99127d4b97cb548,bbb7f25e2ff142740d7cb9f5d971432d,1052067367,0,2918815340,1426902911,680054,30
2758416,1,06a80c4a35c223734cd3f472e1c3d285,9de1665e6430fa58785027dbf9ab53e0,58655385700ba140fd860b23e2e1e9f6,ba195dd85894d6957bc61edca5ac7026,3eaec7d217009d8346b865507a01f5e1,1531543764,0,750787734,1426947290,2758416,30
1169529,0,3e9b0f6563536090873992088b64b9ee,9de1665e6430fa58785027dbf9ab53e0,58655385700ba140fd860b23e2e1e9f6,ba195dd85894d6957bc61edca5ac7026,a487702adcbb4dbdc6ee6830be69ce94,3465302719,2484062282,1428404420,1426895465,1169529,30
273823,0,b5afeaf15ebd48084b163cdf7cba75cd,9a6e6cc222b26cc083646e13c32730ab,93fa8dfbc9d6d34108b3ae23af966003,3b5a74dc6626c816025680e9c37da348,30486b81c87cde496cb665a1b3a608ad,2779388063,0,2779572644,1426881838,273823,30
...,...,...,...,...,...,...,...,...,...,...,...,...
507873,0,df6f715b3f367a644e8af867a4a63be8,9de1665e6430fa58785027dbf9ab53e0,58655385700ba140fd860b23e2e1e9f6,ba195dd85894d6957bc61edca5ac7026,2187264c7dec9b690141678ad687cfa1,3049285783,0,1418882033,1426905251,507873,30
2578255,0,8d2efe1ce7b6015a159702a906b4800c,9de1665e6430fa58785027dbf9ab53e0,58655385700ba140fd860b23e2e1e9f6,e49dc1e8e576ff35a07e1182f4e98c91,7edc872cd2f06c404db8de25aedbd2a2,3316425350,2380283667,795955078,1426886845,2578255,30
2918244,0,f09d63575b2bbb1c52f942448775f9f0,fce8cc8ce0587de13c3fecbc13537746,58655385700ba140fd860b23e2e1e9f6,e49dc1e8e576ff35a07e1182f4e98c91,c56f364d35ac00627fb6e7ac14374b25,4069826026,0,4036996937,1426895211,2918244,30
214589,0,5bcdb6953984d813eccf4b43448d4e49,932a5cc56763396c6b205a63868d6316,58655385700ba140fd860b23e2e1e9f6,21032c3a66d86aa1d99127d4b97cb548,790fe7d6f0020de9ef593866eae87de7,1116119361,0,1419611539,1426946889,214589,30


In [35]:
d = d.groupby('did').agg({'label':lambda x: list(x), 'vid': lambda x: list(x)}).reset_index()

,did,label,vid
0,0000aa68cb571181da9ce69d451e67c9,[1],[2978668349]
1,0002a1d83ecdb9cd70518fd350c5ace4,"[0, 0, 0]","[2713919673, 4057482096, 2350055813]"
2,0002a9e5f68969e3d488699dfd1ac76c,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[4268649635, 3419976753, 3804715216, 189931573..."
3,0003c32bfde744bd2aabee5b18f9ced1,"[0, 0, 0, 0, 0, 0]","[1910290931, 1907952065, 895468396, 4154943341..."
4,0003fcd46214c18d631714cb31593fec,"[0, 1, 0, 0, 0]","[876578304, 797346696, 2571930839, 2572820506,..."
...,...,...,...
84407,fffbd8dd10d753ac0ed85549909d80af,[0],[3617820734]
84408,fffcb7794e1e7496f687c52429fd8c05,"[0, 0]","[2423340274, 2090774245]"
84409,fffda6aa89e05634706b97f065976baf,"[0, 0, 0]","[1933869142, 4265770661, 708026152]"
84410,fffde0944591a1e0391fffd795ec6ec4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[4208764749, 761399570, 1817439707, 2261765855..."
